In [1]:
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.ndimage
import skimage
from skimage import io
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_ubyte

import cv2
from albumentations import (
    Compose, HorizontalFlip, ShiftScaleRotate, ElasticTransform,
    RandomBrightness, RandomContrast, RandomGamma, CLAHE
)

import sklearn
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import telebot
import time
import tensorflow as tf
from tensorflow import keras

import lime
from lime import lime_image

import pandas as pd
import pickle

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
  try:
    print("gpus existem")
    print(gpus)
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)
    
    
TELEBOT_TOKEN = "2058519653:AAG5Kf0Othtye8e13F5WPnBQQSdoCt47ifA"

bot = telebot.TeleBot("2058519653:AAG5Kf0Othtye8e13F5WPnBQQSdoCt47ifA")
bot.config['api_key'] = TELEBOT_TOKEN
bot.get_me()

gpus existem
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


{'ok': True,
 'result': {'id': 2058519653,
  'is_bot': True,
  'first_name': 'cnn_covid',
  'username': 'cnn_covid_bot',
  'can_join_groups': True,
  'can_read_all_group_messages': False,
  'supports_inline_queries': False}}

## Augmentation Class

In [2]:
class AugmentationSequence(keras.utils.Sequence):
  def __init__(self, x_set, y_set, batch_size, augmentations):
    self.x, self.y = x_set, y_set
    self.batch_size = batch_size
    self.augment = augmentations

  def __len__(self):
    return int(np.ceil(len(self.x) / float(self.batch_size)))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
    batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
    
    aug_x = np.zeros(batch_x.shape)
    for idx in range(batch_x.shape[0]):
      aug = self.augment(image = batch_x[idx,:,:])
      aug_x[idx,:,:] = aug["image"]

    return np.stack((aug_x,) * 3, axis = -1), batch_y


## Load Datasets

In [3]:
file_pi = open('input/train_dataset.pkl', 'rb') 
train_generator =  pickle.load(file_pi)

X_val = np.load('input/X_val.npy')
Y_val = np.load('input/Y_val.npy')

steps_per_epoch = 321

In [4]:
type(train_generator)

__main__.AugmentationSequence

## Model Finetune

In [5]:
np.random.seed(587)

tf.keras.backend.clear_session()
model_name = "cache/tl_vgg16_finetune_cd.h5"
model = keras.models.load_model(model_name)

model.summary()



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 9, 9, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              104960

In [6]:
checkpointer = keras.callbacks.ModelCheckpoint(
  model_name,
  monitor = "val_accuracy",
  verbose = 1, 
  save_best_only = True
)



reduce_learning_rate = keras.callbacks.ReduceLROnPlateau(
  monitor = "loss", 
  factor = 0.5, 
  patience = 3, 
  verbose = 1
)

start = time.time()    
fit = model.fit(train_generator, 
    steps_per_epoch = steps_per_epoch, 
    epochs = 100,
    validation_data = (X_val, Y_val),
    callbacks = [
      checkpointer,
      reduce_learning_rate
    ]
  )

end = time.time()

final_train1 = end-start

Epoch 1/100
321/321 [==============================] - 141s 391ms/step - loss: 0.7100 - accuracy: 0.7306 - val_loss: 3.7639 - val_accuracy: 0.5246

Epoch 00001: val_accuracy improved from -inf to 0.52455, saving model to cache/tl_vgg16_finetune_cd.h5
Epoch 2/100
321/321 [==============================] - 111s 345ms/step - loss: 0.5104 - accuracy: 0.8005 - val_loss: 1.8863 - val_accuracy: 0.6329

Epoch 00002: val_accuracy improved from 0.52455 to 0.63289, saving model to cache/tl_vgg16_finetune_cd.h5
Epoch 3/100
321/321 [==============================] - 112s 347ms/step - loss: 0.4360 - accuracy: 0.8310 - val_loss: 3.9788 - val_accuracy: 0.3235

Epoch 00003: val_accuracy did not improve from 0.63289
Epoch 4/100
321/321 [==============================] - 112s 349ms/step - loss: 0.4038 - accuracy: 0.8522 - val_loss: 0.6609 - val_accuracy: 0.7194

Epoch 00004: val_accuracy improved from 0.63289 to 0.71941, saving model to cache/tl_vgg16_finetune_cd.h5
Epoch 5/100
321/321 [=================

321/321 [==============================] - 111s 344ms/step - loss: 0.1177 - accuracy: 0.9587 - val_loss: 0.3489 - val_accuracy: 0.9041

Epoch 00040: val_accuracy did not improve from 0.92829
Epoch 41/100
321/321 [==============================] - 111s 344ms/step - loss: 0.1186 - accuracy: 0.9610 - val_loss: 0.7884 - val_accuracy: 0.7724

Epoch 00041: val_accuracy did not improve from 0.92829
Epoch 42/100
321/321 [==============================] - 109s 340ms/step - loss: 0.0907 - accuracy: 0.9719 - val_loss: 0.4076 - val_accuracy: 0.8730

Epoch 00042: val_accuracy did not improve from 0.92829
Epoch 43/100
321/321 [==============================] - 110s 341ms/step - loss: 0.1127 - accuracy: 0.9614 - val_loss: 0.2434 - val_accuracy: 0.9252

Epoch 00043: val_accuracy did not improve from 0.92829
Epoch 44/100
321/321 [==============================] - 110s 343ms/step - loss: 0.0994 - accuracy: 0.9659 - val_loss: 0.3987 - val_accuracy: 0.8932

Epoch 00044: val_accuracy did not improve from 0


Epoch 00077: val_accuracy did not improve from 0.94310
Epoch 78/100
321/321 [==============================] - 110s 341ms/step - loss: 0.0083 - accuracy: 0.9982 - val_loss: 0.2966 - val_accuracy: 0.9415

Epoch 00078: val_accuracy did not improve from 0.94310
Epoch 79/100
321/321 [==============================] - 111s 345ms/step - loss: 0.0087 - accuracy: 0.9977 - val_loss: 0.3135 - val_accuracy: 0.9353

Epoch 00079: val_accuracy did not improve from 0.94310
Epoch 80/100
321/321 [==============================] - 110s 344ms/step - loss: 0.0082 - accuracy: 0.9979 - val_loss: 0.3213 - val_accuracy: 0.9376

Epoch 00080: val_accuracy did not improve from 0.94310

Epoch 00080: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-06.
Epoch 81/100
321/321 [==============================] - 110s 342ms/step - loss: 0.0045 - accuracy: 0.9992 - val_loss: 0.3020 - val_accuracy: 0.9415

Epoch 00081: val_accuracy did not improve from 0.94310
Epoch 82/100
321/321 [==========================

### Save Time

In [7]:
text_file = open("input/time_train_2.txt", "wt")
n = text_file.write(str(final_train1))
text_file.close()